# Koog A2A Client Notebook

This notebook demonstrates how to create an Agent-to-Agent (A2A) client and interact with an A2A server using the Koog library. The client will connect to the joke agent server, retrieve its agent card, and ask for a joke.

## 1. Dependencies

This cell declares the necessary dependencies for the A2A client. It includes libraries for the Koog agents, the A2A client, and the HTTP JSON-RPC transport.

In [1]:
// show kernel version
"""
Kotlin Jupyter kernel version: ${notebook.kernelVersion} (Update in Settings > Tools > Kotlin Notebooks > Kernel version)
Java Runtime Environment version: ${notebook.jreInfo.javaVersion}
"""


Kotlin Jupyter kernel version: 0.15.2.706.dev1 (Update in Settings > Tools > Kotlin Notebooks > Kernel version)
Java Runtime Environment version: 21


In [2]:
USE {
    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("ai.koog:koog-agents-jvm:0.5.2")
        implementation("ai.koog:a2a-client-jvm:0.5.2")
        implementation("ai.koog:a2a-transport-client-jsonrpc-http-jvm:0.5.2")

    }
}
notebook.dependencyManager.currentBinaryClasspath.map { it.toPath().fileName }.joinToString("\n")

lib-0.15.2-706-1.jar
protocol-api-0.15.2-706-1.jar
kotlin-reflect-2.2.20.jar
api-0.15.2-706-1.jar
annotations-13.0.jar
kotlinx-serialization-json-jvm-1.9.0.jar
kotlin-stdlib-2.2.20.jar
kotlin-script-runtime-2.2.20.jar
slf4j-api-2.0.17.jar
kotlinx-serialization-core-jvm-1.9.0.jar

## 2. Initialize Client

This cell initializes the A2A client. It sets up the transport layer and the agent card resolver.

- `HttpJSONRPCClientTransport`: This is used to send and receive messages to the A2A server over HTTP using the JSON-RPC protocol.
- `UrlAgentCardResolver`: This is used to fetch the agent card from the server's URL.
- `A2AClient`: The main client object that uses the transport and resolver to communicate with the agent.

In [42]:
import ai.koog.a2a.client.A2AClient
import ai.koog.a2a.client.UrlAgentCardResolver
import ai.koog.a2a.model.Message
import ai.koog.a2a.model.MessageSendParams
import ai.koog.a2a.model.Role
import ai.koog.a2a.model.TextPart
import ai.koog.a2a.transport.Request
import ai.koog.a2a.transport.client.jsonrpc.http.HttpJSONRPCClientTransport
import kotlinx.coroutines.runBlocking
import java.util.UUID

// URL of the A2A server
val a2aServerUrl = "http://localhost:9000/a2a"

// HTTP JSON-RPC transport
val transport = HttpJSONRPCClientTransport(
    url = a2aServerUrl
)

// Agent card resolver
val agentCardResolver = UrlAgentCardResolver(
    baseUrl = a2aServerUrl
)

// Create client
val client = A2AClient(transport, agentCardResolver)

## 3. Get Agent Card

This cell retrieves and prints the agent card from the server. The agent card contains metadata about the agent, such as its name, description, and capabilities. This is useful for discovering what an agent can do.

In [43]:
// Get and print the agent card
runBlocking {
    println(client.getAgentCard())
}


AgentCard(protocolVersion=0.3.0, name=Joke maker, description=AI agent specialized in telling joke, url=http://localhost:9000/a2a, preferredTransport=TransportProtocol(value=JSONRPC), additionalInterfaces=null, iconUrl=null, provider=null, version=2.1.0, documentationUrl=null, capabilities=AgentCapabilities(streaming=true, pushNotifications=true, stateTransitionHistory=true, extensions=null), securitySchemes={}, security=[], defaultInputModes=[text/plain, text/markdown, image/jpeg], defaultOutputModes=[text/plain, text/markdown, application/json], skills=[AgentSkill(id=joke-generation, name=Joke Generation, description=Generate joke based on content, tags=[joke], examples=[Joke about apple], inputModes=null, outputModes=null, security=null)], supportsAuthenticatedExtendedCard=true, signatures=null)


## 4. Send Message

This cell sends a message to the agent and prints the response. It creates a `Message` object with a prompt, sends it to the server using `client.sendMessage`, and then prints the agent's reply.

In [45]:
val prompt = "joke about apple"

runBlocking {
    // Create a message to send to the agent
    val message = Message(
        messageId = UUID.randomUUID().toString(),
        contextId = UUID.randomUUID().toString(),
        role = Role.User,
        parts = listOf(TextPart(prompt))
    )
    println("Sending request with $message")

    // Send the message and get the response
    val response = client.sendMessage(Request(
        MessageSendParams(message)
    ))

    println("Received request with ${response.data}")
}


Sending request with Message(messageId=bf10eafa-2583-4a8e-800f-4ef550f8dcbf, role=User, parts=[TextPart(text=joke about apple, metadata=null)], extensions=null, taskId=null, referenceTaskIds=null, contextId=4d4afb9f-a369-468d-a347-6d5c2ffbab07, metadata=null)
Received request with Message(messageId=4f06d5a0-0eaa-463d-b026-65b7d721ce73, role=Agent, parts=[TextPart(text=Why did the apple go to therapy?

Because it couldn’t stop feeling *peel*ed and wanted to work on its core issues! 🍎, metadata=null)], extensions=null, taskId=null, referenceTaskIds=null, contextId=4d4afb9f-a369-468d-a347-6d5c2ffbab07, metadata=null)
